# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [5]:
base = lambda x: x
def foldl2(f, x0, lst):
    def foo(val_lst, acc):
        def new_f(n):
            return acc(f(n,val_lst))
        return new_f
    return foldr(foo, base, lst)(x0)

In [6]:
base = lambda x: x
def foldr2(f, x0, lst):
    def foo(acc, val_lst):
        def new_f(n):
            return acc(f(val_lst, n))
        return new_f
    return foldl(foo, base, lst)(x0)

In [7]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3]), \
foldr2(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

In [8]:
foldl2(lambda x, y: x + y, 0, [1, 2, 3]), \
foldr2(lambda x, y: x + y, 0, [1, 2, 3])

(6, 6)

In [9]:
foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3]), \
foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

('123', '123')

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [10]:
def do_map(string):
    my_dict = dict()
    for i in range(len(string)):
        if string[i] in list(my_dict.keys()):
            my_dict[string[i]] += 1
        else:
            my_dict[string[i]] = 1
    return my_dict



def check_inv(a, b):
    if (len(a) >= len(b)):
        flag = False
        b_map = do_map(b)
        for i in range(len(a)-len(b)+1):
            if do_map(a[i:i+len(b)]) == b_map:
                flag = True
                break
        return flag
    else:
        return False

In [11]:
a = 'abcrotm'
b = 'bacrot'

In [12]:
check_inv(a,b)

True

### Сложность O((n-m)*m)

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [13]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left:
            yield from self.left
        if not self.left and not self.right:
            yield self.value
        if self.right:
            yield from self.right
    
    def __str__(self):
        return '{}, {}, {}'.format(
          str(self.value),
          str(self.left),
          str(self.right)).replace(', None, None', '').replace(', None', '')
    
    def __repr__(self):
        return '{}({}, {}, {})'.format(self.__class__.__name__,
          repr(self.value),
          repr(self.left),
          repr(self.right)) \
               .replace(', None, None)', ')') \
               .replace(', None)', ')')
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]
tree.__repr__(),\
tree.__str__(),


('Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))', '0, 1, 3, 4, 2')

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [14]:
oper = {
             '+': (1, lambda x, y: x + y),
             '-': (1, lambda x, y: x - y),
             '*': (2, lambda x, y: x * y),
             '/': (2, lambda x, y: x / y)
}



In [15]:

def get_simbol(s):
    num = ''
    for i in s:
        if i in '0123456789':
            num += i
        elif num:
            yield float(num)
            num = ''
        if (i in oper) or (i in "()"):
            yield i
    if num:
        yield float(num)


In [16]:
def change(var):
    stack = []
    for sm in var:
        if sm in oper:
            while (stack) and (stack[-1] != "(") and (oper[sm][0] <= oper[stack[-1]][0]):
                yield stack.pop()
            stack.append(sm)
        elif sm == ")":
            while stack:
                x = stack.pop()
                if x == "(":
                    break
                yield x
        elif sm == "(":
            stack.append(sm)
        else:
            yield sm
    while stack:
        yield stack.pop()


In [17]:
def calc(ex):
    x = get_simbol(ex)
    pol = change(x)
    stack = []
    for sm in pol:
        if sm in oper:
            a = stack.pop()
            b = stack.pop()
            stack.append(oper[sm][1](b, a))
        else:
            stack.append(sm)
    return stack[0]

In [18]:
a = '2 * (15 - 3 * 4) - 2'

In [19]:
calc(a)

4.0

In [20]:
b = '(1 + 3 * 4) * 3 / 2'

In [21]:
calc(b)

19.5